In [ ]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

In [2]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from pathlib import Path
from urllib.error import HTTPError

ModuleNotFoundError: No module named 'seaborn'

## Filter/Parse the Data 

In [ ]:
def extract_column(dataset, attr, *, pattern=''):
    """
    Returns a list samples of an attribute and
    constrained by some regular expression.
    
    Positional Arguments:
    dataset -- a csv file
    attr -- an attribute of the dataset 
    
    Keyword Arguments (Required):
    regex: the key used to filter specific datapoints (defaults to all)
    """
    df = pd.read_csv(dataset)[attr].squeeze()
    df.replace('/', '-', regex=True, inplace=True)
    return [dp[0] for dp in df.str.findall(fr'^{pattern}.*') if len(dp)]

## Download (Collect/Join), Save, and Load the Data

In [ ]:
repos = (r'https://www.batteryarchive.org/data/' + _ + r'_timeseries.csv'
         for _ in extract_column('metadata.csv', 'cell_id', pattern='SNL'))

def load_battery_data(path):
    """
    Downloads the battery data, encoded as a pickle object (*.pkl)
    into the datasets directory.
    
    Data source -- @www.batteryarchive.org
    """
    
    try:
        os.chdir(path / r'483_ML_BatteryLife/')
        data_path = r'datasets/battery.pkl'
        if not Path('datasets/').is_dir():
            Path("datasets").mkdir(parents=True, exist_ok=True)
        if not Path(data_path).is_file():
            df = pd.concat([pd.read_csv(url) for url in repos])
            df.to_pickle(data_path)
            print("Download Complete!")
        return pd.read_pickle(data_path)
    except HTTPError:
        print("Download Failed!")

In [ ]:
battery = load_battery_data(Path(r'/home/jason/'))  # change path to Github repository

# Look at the Data

In [ ]:
battery.info(show_counts=True)

In [ ]:
battery.head()

In [ ]:
battery.describe()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=10)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=8)
plt.rc('ytick', labelsize=8)

battery.hist(bins=50, figsize=(10, 10))
plt.show()

In [ ]:
from data_visualization import insights_of_df

insights_of_df(battery)

In [ ]:
battery['Date_Time']

In [ ]:
#plt.figure(figsize=(10, 7))
#plt.boxplot(battery['Date_Time'])
#plt.show()

In [ ]:
plt.figure(figsize=(15.8))
